<a href="https://colab.research.google.com/github/guarnieri0/GAIN/blob/master/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import librerie**

In [172]:
import pandas as pd 

import json
import datetime
from sklearn.preprocessing import LabelEncoder
import numpy as np
from numpy.linalg import norm
from numpy import savetxt
from sklearn import preprocessing

**Reading Dataset**

In [162]:
df = pd.read_json("today_dataset.json")
df_past = pd.read_json("past_dataset.json")

**Deleting useless columns**

In [150]:
df = df.drop(['idx','testArgs','lastFailJob','engine','assignee','lastPassJob','projectIdx','validationIdx','templateIdx','triggerIdx','firstFailJob','lastFailJob','silent','status','reportLink'], axis = 1)
df_past = df_past.drop(['idx','testArgs','lastFailJob','engine','assignee','lastPassJob','projectIdx','validationIdx','templateIdx','triggerIdx','firstFailJob','lastFailJob','silent','status','reportLink'], axis = 1)

In [179]:
ts =  pd.Timestamp(year = 2022, month = 10, day = 6, hour = 10, second = 49)


prova = df_past.loc[lambda df: df_past['timestamp'] < ts]

In [180]:
print(prova)

       idx                                           testName testArgs  \
8    64252      \TestSuite\Draco\PSA\PSA_51_SL2PadDummyAPL.py            
9    64241  \TestSuite\Dorado\ShareBuffer\SwapBuffer\SwapB...            
10   64239  \TestSuite\MultiProduct\FBS\FBS_01_CheckFragme...            
11   64234  \TestSuite\Draco\SecureBoot\SecureBoot_02_tamp...            
12   64233  \TestSuite\Dorado\ErrorManagement\GCBackup\GCB...            
..     ...                                                ...      ...   
740  22008  \TestSuite\Generic\WriteReadVerify\WRV_71_Writ...     None   
741  22776  \TestSuite\Generic\RTG\RTG_02_Full_Functionali...     None   
742  23291  \TestSuite\Generic\RTG\RTG_01_Fast_Focus_Mode_...     None   
743  23292  \TestSuite\MultiProduct\LibTest\VuLibTest\VuCm...     None   
744  23294  \TestSuite\Generic\RTG\RTG_01_Fast_Focus_Mode_...     None   

                                           compareInfo      assignee engine  \
8    {"TestArgs": "", "TestName"

In [169]:
print(type(df.loc[0]['timestamp']))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [ ]:
print(df_past)
print(df)

**Retrieving the Error Code and Error description**

In [152]:
df_splitted= df['compareInfo'].str.split(pat=',', expand=True)
df_splitted = df_splitted.drop([0,1], axis = 1)
error_code = df_splitted.iloc[:,0].to_list()
error_description = df_splitted.iloc[:,1].to_list()
df.insert(5, 'error_description', error_description)
df.insert(6,'error_code',error_code)

df_splitted_past= df_past['compareInfo'].str.split(pat=',', expand=True)
df_splitted_past = df_splitted_past.drop([0,1], axis = 1)
error_code = df_splitted_past.iloc[:,0].to_list()
error_description = df_splitted_past.iloc[:,1].to_list()
df_past.insert(5, 'error_description', error_description)
df_past.insert(6,'error_code',error_code)

**Retrieving the test category**

In [ ]:
testname_splitted = df['testName'].str.split(pat='\\', expand = True)
testname_splitted = testname_splitted.drop([0,1,2,4,5], axis = 1)
print(testname_splitted)
testname = testname_splitted.iloc[:,0].to_list()
df.insert(7,'cat_name', testname)
print(df)

testname_past_splitted = df_past['testName'].str.split(pat='\\', expand = True)
testname_past_splitted = testname_past_splitted.drop([0,1,2,4,5], axis = 1)
print(testname_past_splitted)
testname = testname_past_splitted.iloc[:,0].to_list()
df_past.insert(7,'cat_name', testname)
print(df_past)


**Deleting redundant columns**

In [154]:
df = df.drop(['compareInfo','reportTitle'], axis = 1)
df_past = df_past.drop(['compareInfo','reportTitle'], axis = 1)

**Convert timestamp in year/month/day**

---



In [155]:
df['timestamp'] = df['timestamp'].dt.date
df_past['timestamp'] = df_past['timestamp'].dt.date


In [156]:
print(df.columns)
print(df_past.columns)

Index(['testName', 'reportId', 'timestamp', 'error_description', 'error_code',
       'cat_name'],
      dtype='object')
Index(['testName', 'reportId', 'timestamp', 'error_description', 'error_code',
       'cat_name'],
      dtype='object')


**Filling the NULL value**

---

In [157]:
df['error_description'] = df['error_description'].fillna('None')
df['error_code'] = df['error_code'].fillna('None')
#df['error_description'] = df['error_description'].fillna('None')


**Encoding text features**

In [158]:
encoder_df_error_code = LabelEncoder()
#encoder_df_past_error_code = LabelEncoder()
encoder_df_error_description = LabelEncoder()
#encoder_df_past_error_description = LabelEncoder()
encoder_df_cat_name = LabelEncoder()
#encoder_df_past_cat_name = LabelEncoder()


df['error_code'] = encoder_df_error_code.fit_transform(df['error_code'])
df['error_description'] = encoder_df_error_description.fit_transform(df['error_description'])
df['cat_name'] = encoder_df_cat_name.fit_transform(df['cat_name'])

df_past['error_code'] = encoder_df_error_code.fit_transform(df_past['error_code'])
df_past['error_description'] = encoder_df_error_description.fit_transform(df_past['error_description'])
df_past['cat_name'] = encoder_df_cat_name.fit_transform(df_past['cat_name'])


In [160]:
print(df_past.max())
print(df.max())

testName             test_verification
reportId               MNDRACOVAL-9255
timestamp                   2022-10-06
error_description                   32
error_code                          19
cat_name                            76
dtype: object
testName             \TestSuite\Generic\VoltageDetect\BO_06_AqlyVcc...
reportId                                              MNDRACOVAL-38286
timestamp                                                   2022-10-07
error_description                                                    1
error_code                                                           3
cat_name                                                             4
dtype: object


**Calculate the similarity matrix**

In [ ]:
similarity = np.empty((df_past.shape[0],df.shape[0]))
for i in range (0,df_past.shape[0],1):
  for j in range (0, df.shape[0],1):
   
    past_vector = np.array([df_past.iloc[i][3],df_past.iloc[i][4],df_past.iloc[i][5]])
    today_vector = np.array([df.iloc[j][3],df.iloc[j][4],df.iloc[j][5]])
    print(today_vector)

    #normalized_past_arr = preprocessing.normalize(past_vector)
    #normalized_today_arr = preprocessing.normalize(today_vector)
    
    #print(normalized_past_arr)
    #print(normalized_today_arr)
    #cosin similarity
    similarity[i,j] = np.dot(past_vector,today_vector)/(norm(past_vector)*norm(today_vector))
  

In [140]:
print(similarity)

[[0.98437404 0.94515531 0.46070388 0.15749985 0.95088698]
 [0.96405176 0.96658021 0.55204366 0.26055453 0.97879763]
 [0.98362124 0.94443251 0.4769408  0.174866   0.95798004]
 ...
 [0.99882062 0.8907952  0.33645404 0.02171349 0.90308289]
 [0.96296296 0.96985729 0.55047056 0.25925926 0.97724452]
 [0.99694225 0.90803743 0.37694228 0.06501797 0.92076918]]


In [108]:
savetxt('data.csv', similarity, delimiter = ',')

In [144]:
#max_position=np.amax(similarity.iloc[],axis = 0)
#print(max_position)
res = np.where(similarity[:,0] == np.amax(similarity[:,0]))
print(res)


for i in res:
  print(df_past.iloc[i])
print(df)

(array([189]),)
                                              testName          reportId  \
189  \TestSuite\MultiProduct\SecureWP\SecureWP_25_A...  MNDRACOVAL-18031   

      timestamp  error_description  error_code  cat_name  
189  2022-01-26                  2           1        59  
                                            testName          reportId  \
0  \TestSuite\Generic\RTG\RTG_02_Full_Functionali...  MNDRACOVAL-38286   
1  \TestSuite\Generic\VoltageDetect\BO_06_AqlyVcc...  MNDRACOVAL-38285   
2        \TestSuite\Draco\Latency\Latency_BBM_TLC.py  MNDRACOVAL-38284   
3  \TestSuite\Dorado\ErrorManagement\WriteErrorMa...  MNDRACOVAL-38280   
4  \TestSuite\Draco\QAFW\QAFW_Query_Descriptor_01.py  MNDRACOVAL-38278   

    timestamp  error_description  error_code  cat_name  
0  2022-10-07                  0           0         3  
1  2022-10-07                  1           2         4  
2  2022-10-07                  0           3         1  
3  2022-10-07                  0        

In [147]:
a = [0,0,3]
b = [2,1,59]
print(norm(a))
print(norm(b))
np.dot(a,b)/(norm(a)*norm(b))

3.0
59.04235767650204


0.9992825883286347